In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTEN

smoten_knn_contin = pd.read_csv('smoten_knn_contin.csv', index_col=False)
smoten_median_imputed_contin = pd.read_csv('smoten_median_imputed_contin.csv', index_col=False)

pd.set_option("display.max_columns", None)

def classifyScale_coef(scaleFunc1, estimatorFunc2, k_fold_int, x_array, y_array):
    k = k_fold_int
    kf = KFold(n_splits=k, shuffle=True, random_state=None)    
    clf = make_pipeline(scaleFunc1, estimatorFunc2)

    acc_score = []
    for train_index , test_index in kf.split(X):
        X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train , y_test = y[train_index] , y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
     
    acc = accuracy_score(y_pred , y_test)
    acc_score.append(acc)
     
    avg_acc_score = sum(acc_score)/k
    print('accuracy of each fold - {}'.format(acc_score))
    print('Avg accuracy : {}'.format(avg_acc_score))

    outcome_labels = ['Intubation False', 'Intubation True']

    #print('parameters \n', clf.get_params())
    print('score \n', clf.score(X_train, y_train))
    print('coef_')
    print(clf[1].coef_)
    print('intercept_')
    print(clf[1].intercept_)
    print('fit')
    print(clf.fit(X_train, y_train).n_features_in_, clf.fit(X_train, y_train).feature_names_in_)
    print('decision function \n', clf[1].decision_function(X_test))
    print('classification report \n', classification_report(y_test, y_pred, target_names=outcome_labels))
    micro_roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class="ovr", average="micro")
    print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")
    print('\n')

    

print('### SMOTEN knn contin ###')
print()


print('linear SGDClassifier SVM StandardScaler')
print()
X = smoten_knn_contin.loc[:, ~smoten_knn_contin.columns.isin(['sbp_min', 'sbp_max', 'dbp_max', 'wbc_min', 'creatinine_max', 'hemoglobin_max', 'pt_max', 'outcome'])]
y = smoten_knn_contin['outcome']
sgdClass = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, 
                         l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                         tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                         random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, 
                         early_stopping=True, validation_fraction=0.1, 
                         n_iter_no_change=5, class_weight=None, warm_start=False, average=False)

classifyScale_coef(StandardScaler(), sgdClass, 5, X, y)

print('linear SGDClassifier SVM RobustScaler')
print()

X = smoten_knn_contin.loc[:, ~smoten_knn_contin.columns.isin(['heart_rate_min', 'mbp_min', 'sbp_max', 'dbp_max', 'temperature_min', 'temperature_max', 'creatinine_min','outcome'])]
y = smoten_knn_contin['outcome']
sgdClass = SGDClassifier(loss='hinge', penalty='l2', alpha=0.1001, 
                         l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                         tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                         random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, 
                         early_stopping=True, validation_fraction=0.5, 
                         n_iter_no_change=5, class_weight=None, warm_start=False, average=False)


classifyScale_coef(RobustScaler(), sgdClass, 5, X, y)


print('### SMOTEN median contin ###')
print()


print('linear SGDClassifier SVM StandardScaler')
print()
X = smoten_median_imputed_contin.loc[:, ~smoten_median_imputed_contin.columns.isin(['heart_rate_min', 'mbp_max', 'mbp_mean', 'sbp_max', 'dbp_min', 'dbp_max', 'glucose_min','outcome'])]
y = smoten_median_imputed_contin['outcome']

sgdClass = SGDClassifier(loss='hinge', penalty='l2', alpha=0.1001, 
                         l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                         tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                         random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, 
                         early_stopping=True, validation_fraction=0.7, 
                         n_iter_no_change=5, class_weight=None, warm_start=False, average=False)

                                                             
classifyScale_coef(StandardScaler(), sgdClass, 5, X, y)

print('linear SGDClassifier SVM RobustScaler')
print()
X = smoten_median_imputed_contin.loc[:, ~smoten_median_imputed_contin.columns.isin(['heart_rate_min', 'heart_rate_max', 'mbp_max', 'mbp_mean', 'sbp_max', 'dbp_min', 'dbp_max', 'temperature_min', 'glucose_max', 'wbc_min', 'creatinine_max', 'gender','outcome'])]
y = smoten_median_imputed_contin['outcome']

sgdClass = SGDClassifier(loss='hinge', penalty='l2', alpha=0.1001, 
                         l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                         tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                         random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, 
                         early_stopping=True, validation_fraction=0.1, 
                         n_iter_no_change=5, class_weight=None, warm_start=False, average=False)

classifyScale_coef(RobustScaler(), sgdClass, 5, X, y)




### SMOTEN knn contin ###

linear SGDClassifier SVM StandardScaler

accuracy of each fold - [0.9598976109215017]
Avg accuracy : 0.19197952218430034
score 
 0.9602175767918089
coef_
[[-0.88451962  0.17574613  0.14865798 -0.44987056 -0.08853452 -0.06603451
   0.55862028 -0.4671702  -0.09301002 -0.82486062 -0.01101034  0.04572543
  -0.4895547  -0.16641235 -0.3194498  -0.29962067 -0.51812031 -0.38111798
  -0.26880309 -0.90124567  0.04124615]]
intercept_
[-0.21192094]
fit
21 ['age' 'heart_rate_min' 'heart_rate_max' 'heart_rate_mean' 'mbp_min'
 'mbp_max' 'mbp_mean' 'sbp_mean' 'dbp_min' 'dbp_mean' 'temperature_min'
 'temperature_max' 'temperature_mean' 'glucose_min' 'glucose_max'
 'wbc_max' 'creatinine_min' 'hemoglobin_min' 'pt_min' 'urineoutput'
 'gender']
decision function 
 [-1721.38262437  -561.22081804  -967.03431156 ... -1071.66371395
  -766.81611898 -1223.92965955]
classification report 
                   precision    recall  f1-score   support

Intubation False       0.96      0.96  

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


21 ['age' 'heart_rate_max' 'heart_rate_mean' 'mbp_max' 'mbp_mean' 'sbp_min'
 'sbp_mean' 'dbp_min' 'dbp_mean' 'temperature_mean' 'glucose_min'
 'glucose_max' 'wbc_min' 'wbc_max' 'creatinine_max' 'hemoglobin_min'
 'hemoglobin_max' 'pt_min' 'pt_max' 'urineoutput' 'gender']
decision function 
 [-1744.36265506  -311.88271398  -525.91047803 ...  -290.74475843
  -589.21751477  -596.22231853]
classification report 
                   precision    recall  f1-score   support

Intubation False       0.96      0.97      0.96      7061
 Intubation True       0.97      0.96      0.96      7003

        accuracy                           0.96     14064
       macro avg       0.96      0.96      0.96     14064
    weighted avg       0.96      0.96      0.96     14064

Micro-averaged One-vs-Rest ROC AUC score:
0.96


### SMOTEN median contin ###

linear SGDClassifier SVM StandardScaler

accuracy of each fold - [0.9601820250284414]
Avg accuracy : 0.19203640500568828
score 
 0.9623506825938567
coef_
[[-0

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


21 ['age' 'heart_rate_max' 'heart_rate_mean' 'mbp_min' 'sbp_min' 'sbp_mean'
 'dbp_mean' 'temperature_min' 'temperature_max' 'temperature_mean'
 'glucose_max' 'wbc_min' 'wbc_max' 'creatinine_min' 'creatinine_max'
 'hemoglobin_min' 'hemoglobin_max' 'pt_min' 'pt_max' 'urineoutput'
 'gender']
decision function 
 [-1011.16914957  -893.60629268  -816.5344071  ...  -573.53363991
  -515.8247701   -478.90991596]
classification report 
                   precision    recall  f1-score   support

Intubation False       0.95      0.97      0.96      7040
 Intubation True       0.97      0.95      0.96      7024

        accuracy                           0.96     14064
       macro avg       0.96      0.96      0.96     14064
    weighted avg       0.96      0.96      0.96     14064

Micro-averaged One-vs-Rest ROC AUC score:
0.96


linear SGDClassifier SVM RobustScaler

accuracy of each fold - [0.9611063708759955]
Avg accuracy : 0.1922212741751991
score 
 0.9620484926052332
coef_
[[-0.50677332 -0.2

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


16 ['age' 'heart_rate_mean' 'mbp_min' 'sbp_min' 'sbp_mean' 'dbp_mean'
 'temperature_max' 'temperature_mean' 'glucose_min' 'wbc_max'
 'creatinine_min' 'hemoglobin_min' 'hemoglobin_max' 'pt_min' 'pt_max'
 'urineoutput']
decision function 
 [-1050.6867562   -291.5223913  -1013.66563183 ...  -763.66292827
  -509.3814257   -564.20596946]
classification report 
                   precision    recall  f1-score   support

Intubation False       0.96      0.97      0.96      6989
 Intubation True       0.97      0.96      0.96      7075

        accuracy                           0.96     14064
       macro avg       0.96      0.96      0.96     14064
    weighted avg       0.96      0.96      0.96     14064

Micro-averaged One-vs-Rest ROC AUC score:
0.96




F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


In [5]:
print('###SFS otpimised###')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTEN

smoten_knn_contin = pd.read_csv('smoten_knn_contin.csv', index_col=False).drop(['pt_min','pt_max','urineoutput'],axis=1)
smoten_median_imputed_contin = pd.read_csv('smoten_median_imputed_contin.csv', index_col=False)

pd.set_option("display.max_columns", None)

def classifyScale_coef(scaleFunc1, estimatorFunc2, k_fold_int, x_array, y_array):
    k = k_fold_int
    kf = KFold(n_splits=k, shuffle=True, random_state=None)    
    clf = make_pipeline(scaleFunc1, estimatorFunc2)

    acc_score = []
    for train_index , test_index in kf.split(X):
        X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train , y_test = y[train_index] , y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
     
    acc = accuracy_score(y_pred , y_test)
    acc_score.append(acc)
     
    avg_acc_score = sum(acc_score)/k
    print('accuracy of each fold - {}'.format(acc_score))
    print('Avg accuracy : {}'.format(avg_acc_score))

    outcome_labels = ['Intubation False', 'Intubation True']

    #print('parameters \n', clf.get_params())
    print('score \n', clf.score(X_train, y_train))
    print('coef_')
    print(clf[1].coef_)
    print('intercept_')
    print(str(clf[1].intercept_).strip('[').strip(']'))
    print('fit')
    for i in range(len(clf.feature_names_in_)):
            #print(clf.feature_names_in_[i], ":" ,clf[1].coef_[i])
            print(clf.feature_names_in_[i], str(clf[1].coef_).strip('[').strip(']').split()[i])
    print('decision function \n', clf[1].decision_function(X_test))
    print('classification report \n', classification_report(y_test, y_pred, target_names=outcome_labels))
    micro_roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class="ovr", average="micro")
    print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")
    print('\n')


print('### SMOTEN knn contin ###')
print()


print('linear SGDClassifier SVM StandardScaler')
print()
X = smoten_knn_contin[['age', 'heart_rate_mean', 'dbp_mean', 'temperature_mean', 'wbc_max']]
y = smoten_knn_contin['outcome']
sgdClass = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, 
                         l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                         tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                         random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, 
                         early_stopping=True, validation_fraction=0.1, 
                         n_iter_no_change=5, class_weight=None, warm_start=False, average=False)

classifyScale_coef(StandardScaler(), sgdClass, 5, X, y)

print('linear SGDClassifier SVM RobustScaler')
print()

X = smoten_knn_contin[['age', 'dbp_mean', 'temperature_mean', 'glucose_max', 'wbc_max']]
y = smoten_knn_contin['outcome']
sgdClass = SGDClassifier(loss='hinge', penalty='l2', alpha=0.1001, 
                         l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                         tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                         random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, 
                         early_stopping=True, validation_fraction=0.5, 
                         n_iter_no_change=5, class_weight=None, warm_start=False, average=False)


classifyScale_coef(RobustScaler(), sgdClass, 5, X, y)


print('### SMOTEN median contin ###')
print()


print('linear SGDClassifier SVM StandardScaler')
print()
X = smoten_median_imputed_contin[['age', 'dbp_mean', 'temperature_mean', 'glucose_max', 'wbc_max']]
y = smoten_median_imputed_contin['outcome']

sgdClass = SGDClassifier(loss='hinge', penalty='l2', alpha=0.1001, 
                         l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                         tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                         random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, 
                         early_stopping=True, validation_fraction=0.7, 
                         n_iter_no_change=5, class_weight=None, warm_start=False, average=False)

                                                             
classifyScale_coef(StandardScaler(), sgdClass, 5, X, y)

print('linear SGDClassifier SVM RobustScaler')
print()
X = smoten_median_imputed_contin[['age','dbp_mean','temperature_mean','glucose_max','wbc_max']]
y = smoten_median_imputed_contin['outcome']

sgdClass = SGDClassifier(loss='hinge', penalty='l2', alpha=0.1001, 
                         l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
                         tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                         random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, 
                         early_stopping=True, validation_fraction=0.1, 
                         n_iter_no_change=5, class_weight=None, warm_start=False, average=False)

classifyScale_coef(RobustScaler(), sgdClass, 5, X, y)




### SMOTEN knn contin ###

linear SGDClassifier SVM StandardScaler

accuracy of each fold - [0.925981228668942]
Avg accuracy : 0.1851962457337884
score 
 0.9272255403868032
coef_
[[-1.30063444 -0.49423296 -1.1437518  -0.67203997 -0.75007933]]
intercept_
-0.13748089
fit
age -1.30063444
heart_rate_mean -0.49423296
dbp_mean -1.1437518
temperature_mean -0.67203997
wbc_max -0.75007933
decision function 
 [-215.52225241 -245.28542661 -227.97480364 ... -188.95747053 -185.61319718
 -191.03170063]
classification report 
                   precision    recall  f1-score   support

Intubation False       0.93      0.92      0.93      7029
 Intubation True       0.92      0.93      0.93      7035

        accuracy                           0.93     14064
       macro avg       0.93      0.93      0.93     14064
    weighted avg       0.93      0.93      0.93     14064

Micro-averaged One-vs-Rest ROC AUC score:
0.93


linear SGDClassifier SVM RobustScaler

accuracy of each fold - [0.9282565415244596

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


accuracy of each fold - [0.9275455062571104]
Avg accuracy : 0.1855091012514221
score 
 0.9276521615472128
coef_
[[-0.65308638 -0.66225317 -0.47111325 -0.39983465 -0.44777694]]
intercept_
-0.06792902
fit
age -0.65308638
dbp_mean -0.66225317
temperature_mean -0.47111325
glucose_max -0.39983465
wbc_max -0.44777694
decision function 
 [-349.00519233 -164.22174691 -160.53906429 ... -127.41544647 -128.45974066
 -121.13709557]
classification report 
                   precision    recall  f1-score   support

Intubation False       0.94      0.92      0.93      7037
 Intubation True       0.92      0.94      0.93      7027

        accuracy                           0.93     14064
       macro avg       0.93      0.93      0.93     14064
    weighted avg       0.93      0.93      0.93     14064

Micro-averaged One-vs-Rest ROC AUC score:
0.93


linear SGDClassifier SVM RobustScaler

accuracy of each fold - [0.9276877133105802]
Avg accuracy : 0.18553754266211603
score 
 0.9279543515358362
coef_


F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


no smoting only imputed

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTEN

knn_imputed_contin = pd.read_csv('knn_imputed_contin.csv', index_col=False)
median_imputed_contin = pd.read_csv('median_imputed_contin.csv', index_col=False)

pd.set_option("display.max_columns", None)

def classifyScale_coef(scaleFunc1, estimatorFunc2, k_fold_int, x_array, y_array):
    k = k_fold_int
    kf = KFold(n_splits=k, shuffle=True, random_state=None)    
    clf = make_pipeline(scaleFunc1, estimatorFunc2)

    acc_score = []
    for train_index , test_index in kf.split(X):
        X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train , y_test = y[train_index] , y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
     
    acc = accuracy_score(y_pred , y_test)
    acc_score.append(acc)
     
    avg_acc_score = sum(acc_score)/k
    print('accuracy of each fold - {}'.format(acc_score))
    print('Avg accuracy : {}'.format(avg_acc_score))

    outcome_labels = ['Intubation False', 'Intubation True']

    #print('parameters \n', clf.get_params())
    print('score \n', clf.score(X_train, y_train))
    print('coef_')
    print(clf[1].coef_)
    print('intercept_')
    print(clf[1].intercept_)
    print('fit')
    print(clf.fit(X_train, y_train).n_features_in_, clf.fit(X_train, y_train).feature_names_in_)
    print('decision function \n', clf[1].decision_function(X_test))
    print('classification report \n', classification_report(y_test, y_pred, target_names=outcome_labels))
    micro_roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class="ovr", average="micro")
    print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")
    print('\n')

    
print('### encoded knn contin ###')
print()

X = knn_imputed_contin.drop('outcome',axis= 1)
y = knn_imputed_contin['outcome']

sgdClass = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, average=False)
print('linear SGDClassifier SVM StandardScaler')
print()
classifyScale_coef(StandardScaler(), sgdClass, 5, X, y)

print('linear SGDClassifier SVM RobustScaler')
classifyScale_coef(RobustScaler(), sgdClass, 5, X, y)


print('### encoded median contin ###')
print()

X = median_imputed_contin.drop('outcome',axis= 1)
y = median_imputed_contin['outcome']

print('linear SGDClassifier SVM StandardScaler')
print()
classifyScale_coef(StandardScaler(), sgdClass, 5, X, y)

print('linear SGDClassifier SVM RobustScaler')
classifyScale_coef(RobustScaler(), sgdClass, 5, X, y)




### encoded knn contin ###

linear SGDClassifier SVM StandardScaler

accuracy of each fold - [0.9635466630121968]
Avg accuracy : 0.19270933260243936
score 
 0.9635859139490272
coef_
[[ 0.01812914 -0.00472284  0.0127385  -0.02268141  0.02742    -0.0632219
   0.095881   -0.01776602  0.02550444 -0.02228783  0.00429235 -0.04478802
  -0.0829721  -0.03448846 -0.01995357 -0.03389797 -0.04089212 -0.02605259
  -0.00572451 -0.01872697  0.11595168 -0.09904883 -0.06158674  0.0101252
  -0.00538107 -0.06071492 -0.02271902  0.04127327]]
intercept_
[-1.29453647]
fit
28 ['age' 'heart_rate_min' 'heart_rate_max' 'heart_rate_mean' 'mbp_min'
 'mbp_max' 'mbp_mean' 'sbp_min' 'sbp_max' 'sbp_mean' 'dbp_min' 'dbp_max'
 'dbp_mean' 'temperature_min' 'temperature_max' 'temperature_mean'
 'glucose_min' 'glucose_max' 'wbc_min' 'wbc_max' 'creatinine_min'
 'creatinine_max' 'hemoglobin_min' 'hemoglobin_max' 'pt_min' 'pt_max'
 'urineoutput' 'gender']


F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. 

decision function 
 [-221.48578913  -33.88588703 -165.03843495 ...  -11.29224648 -325.11569734
  -99.50132456]
classification report 
                   precision    recall  f1-score   support

Intubation False       0.96      1.00      0.98      7031
 Intubation True       0.00      0.00      0.00       266

        accuracy                           0.96      7297
       macro avg       0.48      0.50      0.49      7297
    weighted avg       0.93      0.96      0.95      7297

Micro-averaged One-vs-Rest ROC AUC score:
0.50


linear SGDClassifier SVM RobustScaler
accuracy of each fold - [0.9628614499109223]
Avg accuracy : 0.19257228998218445
score 
 0.9637571937517128
coef_
[[-0.00834184  0.03065476 -0.00348861 -0.00281682 -0.02401909 -0.05652278
   0.11285132  0.01921104  0.02922279 -0.08674695  0.00672258  0.00875075
  -0.12523405  0.04637015 -0.00027505 -0.00480252 -0.04728039 -0.03116021
   0.01926877 -0.03217226 -0.04926105 -0.03866892 -0.11138507  0.05374817
  -0.03197389 -0.0

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. 

28 ['age' 'heart_rate_min' 'heart_rate_max' 'heart_rate_mean' 'mbp_min'
 'mbp_max' 'mbp_mean' 'sbp_min' 'sbp_max' 'sbp_mean' 'dbp_min' 'dbp_max'
 'dbp_mean' 'temperature_min' 'temperature_max' 'temperature_mean'
 'glucose_min' 'glucose_max' 'wbc_min' 'wbc_max' 'creatinine_min'
 'creatinine_max' 'hemoglobin_min' 'hemoglobin_max' 'pt_min' 'pt_max'
 'urineoutput' 'gender']
decision function 
 [ -4.04379737  -1.00223755 -13.48662808 ... -13.13217061  -6.92824351
 -15.18139254]
classification report 
                   precision    recall  f1-score   support

Intubation False       0.96      1.00      0.98      7040
 Intubation True       0.00      0.00      0.00       257

        accuracy                           0.96      7297
       macro avg       0.48      0.50      0.49      7297
    weighted avg       0.93      0.96      0.95      7297

Micro-averaged One-vs-Rest ROC AUC score:
0.50


linear SGDClassifier SVM RobustScaler
accuracy of each fold - [0.9608058106070988]
Avg accuracy : 

F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. 

28 ['age' 'heart_rate_min' 'heart_rate_max' 'heart_rate_mean' 'mbp_min'
 'mbp_max' 'mbp_mean' 'sbp_min' 'sbp_max' 'sbp_mean' 'dbp_min' 'dbp_max'
 'dbp_mean' 'temperature_min' 'temperature_max' 'temperature_mean'
 'glucose_min' 'glucose_max' 'wbc_min' 'wbc_max' 'creatinine_min'
 'creatinine_max' 'hemoglobin_min' 'hemoglobin_max' 'pt_min' 'pt_max'
 'urineoutput' 'gender']
decision function 
 [23.17392135 30.01458519 36.798592   ... 35.07118447 29.02523709
 32.52388632]
classification report 
                   precision    recall  f1-score   support

Intubation False       0.96      1.00      0.98      7011
 Intubation True       0.00      0.00      0.00       286

        accuracy                           0.96      7297
       macro avg       0.48      0.50      0.49      7297
    weighted avg       0.92      0.96      0.94      7297

Micro-averaged One-vs-Rest ROC AUC score:
0.50




F:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
F:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. 